# ALL CODE TAKEN FROM OPENROUTESERVICES EXAMPLE

## please modify before using. 


## please read: 
https://openrouteservice.org/2187-2/
https://saralgyaan.com/posts/set-passwords-and-secret-keys-in-environment-variables-maclinuxwindows-python-quicktip/
https://openrouteservice.org/dev/#/api-docs/optimization/post

In [2]:
import folium

from openrouteservice import client

In [ ]:
api_key = 'your_key'  # Provide your personal API key
ors = client.Client(key=api_key)
# Set up folium map
map1 = folium.Map(tiles='Stamen Toner', location=([37.738684, -122.450523]), zoom_start=12)

# Set up the apartment dictionary with real coordinates
apartments = {'first': {'location': [-122.430954, 37.792965]},
              'second': {'location': [-122.501636, 37.748653]},
              'third': {'location': [-122.446629, 37.736928]}
              }

# Request of isochrones with 15 minute on foot.
params_iso = {'profile': 'foot-walking',
              'range': [900],  # 900/60 = 15 minutes
              'attributes': ['total_pop']  # Get population count for isochrones
              }

for name, apt in apartments.items():
    params_iso['locations'] = [apt['location']]  # Add apartment coords to request parameters
    apt['iso'] = ors.isochrones(**params_iso)  # Perform isochrone request
    folium.features.GeoJson(apt['iso']).add_to(map1)  # Add GeoJson to map

    folium.map.Marker(list(reversed(apt['location'])),  # reverse coords due to weird folium lat/lon syntax
                      icon=folium.Icon(color='lightgray',
                                       icon_color='#cc0000',
                                       icon='home',
                                       prefix='fa',
                                       ),
                      popup=name,
                      ).add_to(map1)  # Add apartment locations to map

map1

In [ ]:
# Common request parameters
params_poi = {'request': 'pois',
              'sortby': 'distance'}

# POI categories according to
# https://giscience.github.io/openrouteservice/documentation/Places.html
categories_poi = {'kindergarten': [153],
                  'supermarket': [518],
                  'hairdresser': [395]}

for name, apt in apartments.items():
    apt['categories'] = dict()  # Store in pois dict for easier retrieval
    params_poi['geojson'] = apt['iso']['features'][0]['geometry']
    print("\n{} apartment".format(name))

    for typ, category in categories_poi.items():
        params_poi['filter_category_ids'] = category
        apt['categories'][typ] = dict()
        apt['categories'][typ]['geojson'] = ors.places(**params_poi)['features']  # Actual POI request
        print(f"\t{typ}: {len(apt['categories'][typ]['geojson'])}")

In [ ]:
# Set up common request parameters
params_route = {'profile': 'foot-walking',
                'format_out': 'geojson',
                'geometry': 'true',
                'format': 'geojson',
                'instructions': 'false',
                }

# Set up dict for font-awesome
style_dict = {'kindergarten': 'child',
              'supermarket': 'shopping-cart',
              'hairdresser': 'scissors'
              }

# Store all routes from all apartments to POIs
for apt in apartments.values():
    for cat, pois in apt['categories'].items():
        pois['durations'] = []
        for poi in pois['geojson']:
            poi_coords = poi['geometry']['coordinates']

            # Perform actual request
            params_route['coordinates'] = [apt['location'],
                                           poi_coords
                                           ]
            json_route = ors.directions(**params_route)

            folium.features.GeoJson(json_route).add_to(map1)
            folium.map.Marker(list(reversed(poi_coords)),
                              icon=folium.Icon(color='white',
                                               icon_color='#1a1aff',
                                               icon=style_dict[cat],
                                               prefix='fa'
                                               )
                              ).add_to(map1)

            poi_duration = json_route['features'][0]['properties']['summary']['duration']
            pois['durations'].append(poi_duration)  # Record durations of routes

map1

In [ ]:
# Sum up the closest POIs to each apartment
for name, apt in apartments.items():
    apt['shortest_sum'] = sum([min(cat['durations']) for cat in apt['categories'].values()])
    print(f"{name} apartment: {round(apt['shortest_sum'] / 60, 1)} min"
          )